Este notebook analisa a aplicação de uma Rede Neural (Multi-layer Perceptron) a um problema de previsão de séries temporais

Referência:
https://machinelearningmastery.com/how-to-develop-multilayer-perceptron-models-for-time-series-forecasting/


## Preparação dos Dados

Um primeiro passo importante para o uso de redes neurais, ou mesmo outros modelos de aprendizado de máquina em prolemas de previsão de séries temporais é a preparação dos dados. A sequência de valores de uma série deve ser remodelado para ser representada como uma base de dados de um modelo de aprendizado supervisionado.

Ou seja, dada a sequência:



```
[10, 20, 30, 40, 50, 60, 70, 80, 90]
```

Temos que representá-la em função de entradas (x) e saídas (y). Mais especificamente, temos a cada amostra em x os valores dos lags e em y os valores futuros. Portanto, a sequência acima para um problema de previsão que utiliza 3 lags pode ser representada como:



```
X,				y
10, 20, 30		40
20, 30, 40		50
30, 40, 50		60
...
```

Essa conversão pode ser realizada pelo método descrito a seguir, que utiliza o valor de n_lags para definir o número de atributos em x

In [1]:
from numpy import array

def split_sequence(sequence, n_lags):
 X, y = list(), list()
 for i in range(len(sequence)):

  end_ix = i + n_lags
  # Testa se a sequência chegou ao fim
  if end_ix > len(sequence)-1:
    break
  # cria os trechos x e y de cada amostra
  seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

Aplicando o método à sequencia definida anteriormente.

In [2]:
sequencia = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# número de lags
n_lags = 3

X, y = split_sequence(sequencia, n_lags)

for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


## O Modelo MLP

Um primeiro passo é a definição de seua arquitetura, em termos de número de camadas, neurônios e tipos de função de ativação.

O exemplo a seguir utiliza a biblioteca Keras, que define tal arquitetura MLP de maneira sequencial, a partir da inclusão das camadas no modelo.

In [3]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_lags))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

ModuleNotFoundError: No module named 'keras'

O modelo MLP definido acima apresenta uma camada inicial que recebe 3, possui 100 neurônios e como função de ativação a 'Relu'. A camada de saída possui apenas 1 neurônio, o que coincide com o formato de y, correspondente ao valor de previsão em t+1.

A função compile é utilizada para fechar a definição do modelo, e configura também alguns métodos internos utilizados na estimativa dos parâmetros.

Após definido, o modelo pode ser utilizado para treinamento com a função fit().

In [ ]:
model.fit(X, y, epochs=2000, verbose=0)

O parâmetro epochs indica o número de iterações que serão realizadas na base inteira de x e y. Verbose, se diferente de zero, gera um histórico sobre a fase de treinamento do modelo.

Para finalizar o processo, é testada a seguir a função predict(). Neste caso, ela é utilizada para realizar a previsão de um valor futuro em y dados os valores de lags descritos no conjunto de atributos em x.

In [ ]:
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_lags))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[101.565834]]


É importante observar que os valores de y são retornados pelo modelo a dentro de um array. Portanto, para efeito de alguma exibição gráfica ou aplicação a alguma métrica de desempenho, é importante observar tal formatação.